## Ginzaによる極性辞書ベースの感情分析

In [12]:
!pip install -U ginza ja-ginza
!pip install nltk
!pip install neologdn

# reload package
import pkg_resources, imp
imp.reload(pkg_resources)

import spacy
nlp = spacy.load('ja_ginza')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import re
import neologdn

def clean_text(_text):
  # trim url
  _text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', _text)

  # trim contents in brackets
  # not defined

  # trim pattern
  _pattern = '[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”◇ᴗ●↓→♪★⊂⊃※△□◎〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％＞＜]'
  _text = re.sub(_pattern, '', _text)

  # trim number
  _text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', _text)
  _text = re.sub(r'\d+', '0', _text)

  # normalization
  _text = neologdn.normalize(_text)

  return _text

def clean_tokenizer(_text):
  res = []
  _text = clean_text(_text)
  doc = nlp(_text)
  for sentence in doc.sents:
    for token in sentence:
      detail_sentence = []
      detail_sentence.append(token.text)    # 単語
      detail_sentence.append(token.lemma_)  # 原型
      detail_sentence.append(token.pos_)    # 品詞
      res.append(detail_sentence)
  return res

a = clean_tokenizer('<spaCy> はオープンソースの(自然言語処理)ライブラリで-----す。学習済みの統計モデルと3,000,000の単語ベクトルが付属しています。')
print(a)

[['spaCy', 'spacy', 'NOUN'], ['は', 'は', 'ADP'], ['オープンソース', 'オープンソース', 'NOUN'], ['の', 'の', 'ADP'], ['自然言語処理', '自然言語処理', 'ADV'], ['ライブラリ', 'ライブラリ', 'NOUN'], ['です', 'です', 'AUX'], ['学習', '学習', 'NOUN'], ['済み', '済み', 'NOUN'], ['の', 'の', 'ADP'], ['統計', '統計', 'NOUN'], ['モデル', 'モデル', 'NOUN'], ['と', 'と', 'ADP'], ['0', '0', 'NUM'], ['の', 'の', 'ADP'], ['単語', '単語', 'NOUN'], ['ベクトル', 'ベクトル', 'NOUN'], ['が', 'が', 'ADP'], ['付属', '付属', 'VERB'], ['し', 'する', 'AUX'], ['て', 'て', 'SCONJ'], ['い', 'いる', 'VERB'], ['ます', 'ます', 'AUX']]


In [14]:
import json
def load_pn_dict(path):
  with open(path, 'r', encoding = 'utf-8') as f:
    res = json.load(f)
  return res

pn_dict = load_pn_dict('drive/My Drive/kenkyu/data/pn_ja_tohoku.json')
print(pn_dict['幸福'])

{'word': '幸福', 'score': 1}


In [15]:
NEGATION = ['ない', 'ず', 'ぬ']
PN_AMORPHOUS = []
def get_word_pn_score(_word, _dict):
  res = 0
  if _word in _dict:
    res = _dict[_word]['score']
  else :
    res = 0
  return res
print(get_word_pn_score('幸福', pn_dict))

1


In [17]:
import math
# ginza token 
#       index 0 : word
#       index 1 : lemma
#       index 2 : pos tag

def get_document_pn_score(_doc, _dict):
  tot_score = 0
  res = []
  text = ''
  tokens = clean_tokenizer(_doc)
  adv_score = 0

  for i, token in enumerate(tokens):
    score = get_word_pn_score(token[1], _dict)
    text += token[0]

    # 否定語
    if token[1] in NEGATION and 'あるじゃない' not in text:
      tot_score *= -1

    # 副詞で強調
    if adv_score != 0:
      score *= adv_score
      adv_score = 0
    if token[2] == 'ADV':
      adv_score = 1.5

    tot_score += score
    if score != 0:
      res.append([token[1], score])

  if len(res) == 0:
    tot_score = 0
  else :
    tot_score /= len(res)

  return math.tanh(tot_score), res

s, _ = get_document_pn_score('私は幸せだ', pn_dict)
print('score : {}'.format(s))
s, _ = get_document_pn_score('私はなんて不幸だ', pn_dict)
print('score : {}'.format(s))
s, _ = get_document_pn_score('君のこと嫌いだよ', pn_dict)
print('score : {}'.format(s))
s, _ = get_document_pn_score('君のこと嫌いじゃないよ', pn_dict)
print('score : {}'.format(s))
s, _ = get_document_pn_score('君のこと嫌いじゃないわけないよ', pn_dict)
print('score : {}'.format(s))
s, al = get_document_pn_score('私はとっても幸せだ', pn_dict)
print('score : {}'.format(s))
print(al)

score : 0.7615941559557649
score : -0.7615941559557649
score : -0.7615941559557649
score : 0.7615941559557649
score : -0.7615941559557649
score : 0.9051482536448664
[['幸せ', 1.5]]


## fine-turing済みBARTによる感情分析

In [8]:
!pip install transformers[ja]

     |████████████████████████████████| 2.8 MB 6.8 MB/s 
     |████████████████████████████████| 636 kB 66.3 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 895 kB 59.8 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |████████████████████████████████| 490 kB 59.9 MB/s 
     |████████████████████████████████| 47.4 MB 44 kB/s 
     |████████████████████████████████| 13.4 MB 208 kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=f93a0fb33d962671ef033313ee5d3d7179100a419dc8de76ef77d5072ee62d8a
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for unidic: filename=unidic-1.0.3-py3-none-any.whl size=5506 sha256=2fe2636018783067106d8c9cb26a76b370525f4fd490a08c01a742b786c5fa02
  Stored in directory: /root/.cache/pip/wheels/23/30/0b/128289fb595ef4117d2976ffdbef5069ef83be813e88caa0a6
  Created wheel for unid

In [9]:
from transformers import pipeline, AutoModelForSequenceClassification, BertJapaneseTokenizer

# パイプラインの準備
model = AutoModelForSequenceClassification.from_pretrained('daigo/bert-base-japanese-sentiment') 
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
nlp = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)

# 感情分析の実行
print(nlp("私はとっても幸せ"))
print(nlp("私はとっても不幸"))

# 慣用句表現
print(nlp("有る事ない事いわないで"))

# 否定表現が一つずれている
print(nlp("君のこと嫌いだよ")) # nega
print(nlp("君のこと嫌いじゃないよ")) # posi
print(nlp("君のこと嫌いじゃないわけないよ")) # nega
print(nlp("君のこと嫌いじゃないわけないことはないよ")) # posi

# 難しい表現
print(nlp("見た目は悪いのですが、本当に美味しいのでしょうか？")) # neutral
print(nlp("味をもっと改良してほしいんですが。"))  # negative
print(nlp("このラーメンは味の割に値段が高い。")) # negative

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

[{'label': 'ポジティブ', 'score': 0.9420029520988464}]
[{'label': 'ネガティブ', 'score': 0.9948635697364807}]
[{'label': 'ネガティブ', 'score': 0.7158402800559998}]
[{'label': 'ネガティブ', 'score': 0.8742923736572266}]
[{'label': 'ネガティブ', 'score': 0.604921817779541}]
[{'label': 'ポジティブ', 'score': 0.5850330591201782}]
[{'label': 'ネガティブ', 'score': 0.6227003931999207}]
[{'label': 'ポジティブ', 'score': 0.9475049376487732}]
[{'label': 'ポジティブ', 'score': 0.9839153289794922}]
[{'label': 'ポジティブ', 'score': 0.9108796119689941}]


## 参考
- pythonで日本語文の感情分析（＋言語処理の基礎）
https://qiita.com/mtskhs/items/0323f299d03f5b07efdc
- はじめての自然言語処理 第4回 spaCy/GiNZA を用いた自然言語処理 https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part4.html
- 形態素解析前の日本語文書の前処理 (Python) https://ohke.hateblo.jp/entry/2019/02/09/141500
- 日本語自然言語処理で必須の前処理まとめ(Dockerによる環境構築込み) https://qiita.com/Keyskey/items/9f5f6c414e0f89a4f931
- 感情分析に用いる極性辞書を自動生成する https://qiita.com/g-k/items/1b7c765fa6520297ca7c
- 【自然言語処理】感情分析の進め方＆ハマりやすいポイント
https://qiita.com/toshiyuki_tsutsui/items/604f92dbe6e20a18a17e